# Similarity of jobs

#Goal:Similarity of Jobs: Given a job title, find the 5 top jobs that are of a similar nature, based on the job post. 

In [1]:
import os
import json
import tqdm
import re
import string
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
import scipy
import numpy as np
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
import spacy

import warnings
warnings.filterwarnings('ignore')

In [2]:
#nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 

### Read data

In [3]:
df_ori = pd.read_csv('C:\\Users\\deeparam\\Desktop\\DMCA\\Sols_DSP_Capstone_Projects_PS_Chegg_2019\\6. TEXT MINING - ONLINE JOB POSTING\data job posts.csv')

df_ori.head()
print(df_ori.shape)
df = df_ori.drop_duplicates(['jobpost','Title'])
print(df.shape)
print("Removed {0} duplicates (based on jobpost + Title)".format(df_ori.shape[0]-df.shape[0]))

(19001, 24)
(18892, 24)
Removed 109 duplicates (based on jobpost + Title)


In [4]:
df.head()

,jobpost,date,Title,Company,AnnouncementCode,Term,Eligibility,Audience,StartDate,Duration,Location,JobDescription,JobRequirment,RequiredQual,Salary,ApplicationP,OpeningDate,Deadline,Notes,AboutC,Attach,Year,Month,IT
0,AMERIA Investment Consulting Company\nJOB TITL...,"Jan 5, 2004",Chief Financial Officer,AMERIA Investment Consulting Company,NaN,NaN,NaN,NaN,NaN,NaN,"Yerevan, Armenia",AMERIA Investment Consulting Company is seekin...,- Supervises financial management and administ...,"To perform this job successfully, an\nindividu...",NaN,"To apply for this position, please submit a\nc...",NaN,26 January 2004,NaN,NaN,NaN,2004,1,False
1,International Research & Exchanges Board (IREX...,"Jan 7, 2004",Full-time Community Connections Intern (paid i...,International Research & Exchanges Board (IREX),NaN,NaN,NaN,NaN,NaN,3 months,"IREX Armenia Main Office; Yerevan, Armenia \nD...",NaN,NaN,- Bachelor's Degree; Master's is preferred;\n-...,NaN,Please submit a cover letter and resume to:\nI...,NaN,12 January 2004,NaN,The International Research & Exchanges Board (...,NaN,2004,1,False
2,Caucasus Environmental NGO Network (CENN)\nJOB...,"Jan 7, 2004",Country Coordinator,Caucasus Environmental NGO Network (CENN),NaN,NaN,NaN,NaN,NaN,Renewable annual contract\nPOSITION,"Yerevan, Armenia",Public outreach and strengthening of a growing...,- Working with the Country Director to provide...,"- Degree in environmentally related field, or ...",NaN,Please send resume or CV toursula.kazarian@......,NaN,20 January 2004\nSTART DATE: February 2004,NaN,The Caucasus Environmental NGO Network is a\nn...,NaN,2004,1,False
3,Manoff Group\nJOB TITLE: BCC Specialist\nPOSI...,"Jan 7, 2004",BCC Specialist,Manoff Group,NaN,NaN,NaN,NaN,NaN,NaN,"Manila, Philippines",The LEAD (Local Enhancement and Development fo...,- Identify gaps in knowledge and overseeing in...,"- Advanced degree in public health, social sci...",NaN,Please send cover letter and resume to Amy\nPe...,NaN,23 January 2004\nSTART DATE: Immediate,NaN,NaN,NaN,2004,1,False
4,Yerevan Brandy Company\nJOB TITLE: Software D...,"Jan 10, 2004",Software Developer,Yerevan Brandy Company,NaN,NaN,NaN,NaN,NaN,NaN,"Yerevan, Armenia",NaN,- Rendering technical assistance to Database M...,- University degree; economical background is ...,NaN,Successful candidates should submit\n- CV; \n-...,NaN,"20 January 2004, 18:00",NaN,NaN,NaN,2004,1,True


### Filtering selective columns

In [5]:
df_job_related = df[['Title','JobDescription','RequiredQual','JobRequirment']]

In [6]:
df_job_related.head(10)

,Title,JobDescription,RequiredQual,JobRequirment
0,Chief Financial Officer,AMERIA Investment Consulting Company is seekin...,"To perform this job successfully, an\nindividu...",- Supervises financial management and administ...
1,Full-time Community Connections Intern (paid i...,NaN,- Bachelor's Degree; Master's is preferred;\n-...,NaN
2,Country Coordinator,Public outreach and strengthening of a growing...,"- Degree in environmentally related field, or ...",- Working with the Country Director to provide...
3,BCC Specialist,The LEAD (Local Enhancement and Development fo...,"- Advanced degree in public health, social sci...",- Identify gaps in knowledge and overseeing in...
4,Software Developer,NaN,- University degree; economical background is ...,- Rendering technical assistance to Database M...
5,Saleswoman,Saleswoman will sell menswear and accessories.,"- Candidates should be female, 20-30 years old...",NaN
6,Chief Accountant/ Finance Assistant,The Armenian Branch Office of the Open Society...,- University degree in finance/ accounting; \n...,NaN
7,Non-paid part or full time Programmatic Intern,NaN,NaN,NaN
8,Assistant to Managing Director,NaN,- University degree;\n- At least 3 years of ex...,- Coordinating the work of subordinate employe...
9,"Program Assistant (INL), FSN-8; FP-6*",The incumbent assists in coordinating INL-fund...,NOTE: All applicants are instructed to\naddres...,NaN


In [7]:
df_job_related['JobDescription'] = df_job_related['JobDescription'].astype(str)
df_job_related['RequiredQual'] = df_job_related['RequiredQual'].astype(str)
df_job_related['JobRequirment'] = df_job_related['JobRequirment'].astype(str)
df_job_related['Title'] = df_job_related['Title'].astype(str)

In [8]:
df_job_related['JobDescription_token'] = df_job_related['JobDescription'].map(word_tokenize)
df_job_related['RequiredQual_token'] = df_job_related['RequiredQual'].map(word_tokenize)
df_job_related['JobRequirment_token'] = df_job_related['JobRequirment'].map(word_tokenize)

#### Removing punctuations and stopwords from the above tokens

In [9]:
# stopwords and punctuation removal
df_job_related['JobDescription_token'] = df_job_related['JobDescription_token'].apply(lambda x: [item for item in x if item not in stop_words])
df_job_related['JobDescription_token'] = df_job_related['JobDescription_token'].apply(lambda x: [item for item in x if item not in string.punctuation])
df_job_related['RequiredQual_token'] = df_job_related['RequiredQual_token'].apply(lambda x: [item for item in x if item not in stop_words])
df_job_related['RequiredQual_token'] = df_job_related['RequiredQual_token'].apply(lambda x: [item for item in x if item not in string.punctuation])
df_job_related['JobRequirment_token'] = df_job_related['JobRequirment_token'].apply(lambda x: [item for item in x if item not in stop_words])
df_job_related['JobRequirment_token'] = df_job_related['JobRequirment_token'].apply(lambda x: [item for item in x if item not in string.punctuation])

In [10]:
# lowercase
df_job_related['JobDescription_token'] = df_job_related['JobDescription_token'].apply(lambda x: [item.lower() for item in x])
df_job_related['RequiredQual_token'] = df_job_related['RequiredQual_token'].apply(lambda x: [item.lower() for item in x])
df_job_related['JobRequirment_token'] = df_job_related['JobRequirment_token'].apply(lambda x: [item.lower() for item in x])

In [11]:
# stem
df_job_related['JobDescription_token'] = df_job_related['JobDescription_token'].apply(lambda x: [PorterStemmer().stem(item) for item in x])
df_job_related['RequiredQual_token'] = df_job_related['RequiredQual_token'].apply(lambda x: [PorterStemmer().stem(item) for item in x])
df_job_related['JobRequirment_token'] = df_job_related['JobRequirment_token'].apply(lambda x: [PorterStemmer().stem(item) for item in x])

In [12]:
# convert list to string
df_job_related['JobDescription_token_str'] = df_job_related['JobDescription_token'].apply(lambda x: " ".join(x))
df_job_related['RequiredQual_token_str'] = df_job_related['RequiredQual_token'].apply(lambda x: " ".join(x))
df_job_related['JobRequirment_token_str'] = df_job_related['JobRequirment_token'].apply(lambda x: " ".join(x))

In [13]:
df_job_related.head(10)

,Title,JobDescription,RequiredQual,JobRequirment,JobDescription_token,RequiredQual_token,JobRequirment_token,JobDescription_token_str,RequiredQual_token_str,JobRequirment_token_str
0,Chief Financial Officer,AMERIA Investment Consulting Company is seekin...,"To perform this job successfully, an\nindividu...",- Supervises financial management and administ...,"[ameria, invest, consult, compani, seek, chief...","[to, perform, job, success, individu, must, ab...","[supervis, financi, manag, administr, staff, i...",ameria invest consult compani seek chief finan...,to perform job success individu must abl perfo...,supervis financi manag administr staff includ ...
1,Full-time Community Connections Intern (paid i...,nan,- Bachelor's Degree; Master's is preferred;\n-...,nan,[nan],"[bachelor, 's, degre, master, 's, prefer, exce...",[nan],nan,bachelor 's degre master 's prefer excel skill...,nan
2,Country Coordinator,Public outreach and strengthening of a growing...,"- Degree in environmentally related field, or ...",- Working with the Country Director to provide...,"[public, outreach, strengthen, grow, network, ...","[degre, environment, relat, field, 5, year, re...","[work, countri, director, provid, environment,...",public outreach strengthen grow network enviro...,degre environment relat field 5 year relev exp...,work countri director provid environment infor...
3,BCC Specialist,The LEAD (Local Enhancement and Development fo...,"- Advanced degree in public health, social sci...",- Identify gaps in knowledge and overseeing in...,"[the, lead, local, enhanc, develop, health, bc...","[advanc, degre, public, health, social, scienc...","[identifi, gap, knowledg, overse, inform, coll...",the lead local enhanc develop health bcc speci...,advanc degre public health social scienc commu...,identifi gap knowledg overse inform collect fi...
4,Software Developer,nan,- University degree; economical background is ...,- Rendering technical assistance to Database M...,[nan],"[univers, degre, econom, background, plu, exce...","[render, technic, assist, databas, manag, syst...",nan,univers degre econom background plu excel know...,render technic assist databas manag system rea...
5,Saleswoman,Saleswoman will sell menswear and accessories.,"- Candidates should be female, 20-30 years old...",nan,"[saleswoman, sell, menswear, accessori]","[candid, femal, 20-30, year, old, nice-look, e...",[nan],saleswoman sell menswear accessori,candid femal 20-30 year old nice-look exterior...,nan
6,Chief Accountant/ Finance Assistant,The Armenian Branch Office of the Open Society...,- University degree in finance/ accounting; \n...,nan,"[the, armenian, branch, offic, open, societi, ...","[univers, degre, finance/, account, one, year,...",[nan],the armenian branch offic open societi institu...,univers degre finance/ account one year minimu...,nan
7,Non-paid part or full time Programmatic Intern,nan,nan,nan,[nan],[nan],[nan],nan,nan,nan
8,Assistant to Managing Director,nan,- University degree;\n- At least 3 years of ex...,- Coordinating the work of subordinate employe...,[nan],"[univers, degre, at, least, 3, year, experi, r...","[coordin, work, subordin, employe, maintain, l...",nan,univers degre at least 3 year experi relev fie...,coordin work subordin employe maintain liaison...
9,"Program Assistant (INL), FSN-8; FP-6*",The incumbent assists in coordinating INL-fund...,NOTE: All applicants are instructed to\naddres...,nan,"[the, incumb, assist, coordin, inl-fund, law, ...","[note, all, applic, instruct, address, select,...",[nan],the incumb assist coordin inl-fund law enforc ...,note all applic instruct address select criter...,nan


### TF-IDF based similarity

In [14]:
tfidf_model = TfidfVectorizer()
Title_tfidf = tfidf_model.fit_transform(df_job_related['Title'])
JobDescription_tfidf = tfidf_model.fit_transform(df_job_related['JobDescription_token_str'])
RequiredQual_tfidf = tfidf_model.fit_transform(df_job_related['RequiredQual_token_str'])
JobRequirment_tfidf = tfidf_model.fit_transform(df_job_related['JobRequirment_token_str'])

In [15]:
def find_similar_jobs_tfidf(index, top_n = 5):
    Title_tfidf_cosine = cosine_similarity(Title_tfidf[index:index+1], Title_tfidf).flatten()
    JobDescription_tfidf_cosine = cosine_similarity(JobDescription_tfidf[index:index+1], JobDescription_tfidf).flatten()
    RequiredQual_tfidf_cosine = cosine_similarity(RequiredQual_tfidf[index:index+1], RequiredQual_tfidf).flatten()
    JobRequirment_tfidf_cosine = cosine_similarity(JobRequirment_tfidf[index:index+1], JobRequirment_tfidf).flatten()

    average_cosine = (Title_tfidf_cosine + JobDescription_tfidf_cosine + RequiredQual_tfidf_cosine + JobRequirment_tfidf_cosine) / 4.0
    related_docs_indices = [i for i in average_cosine.argsort()[::-1] if i != index]
    return [(index, average_cosine[index]) for index in related_docs_indices][0:top_n]

In [16]:
df_job_related.iloc[0]

Title                                                 Chief Financial Officer
JobDescription              AMERIA Investment Consulting Company is seekin...
RequiredQual                To perform this job successfully, an\nindividu...
JobRequirment               - Supervises financial management and administ...
JobDescription_token        [ameria, invest, consult, compani, seek, chief...
RequiredQual_token          [to, perform, job, success, individu, must, ab...
JobRequirment_token         [supervis, financi, manag, administr, staff, i...
JobDescription_token_str    ameria invest consult compani seek chief finan...
RequiredQual_token_str      to perform job success individu must abl perfo...
JobRequirment_token_str     supervis financi manag administr staff includ ...
Name: 0, dtype: object

In [17]:
for index, score in find_similar_jobs_tfidf(0):
    print(df_job_related.iloc[index])
    print("SIMILARITY SCORE: ", score)

Title                                                 Chief Financial Officer
JobDescription              River Island is seeking a Chief Financial Offi...
RequiredQual                - Master's degree in Management, Finance or Ec...
JobRequirment               - Develop tools and systems to provide critica...
JobDescription_token        [river, island, seek, chief, financi, offic, r...
RequiredQual_token          [master, 's, degre, manag, financ, econom, at,...
JobRequirment_token         [develop, tool, system, provid, critic, financ...
JobDescription_token_str    river island seek chief financi offic respons ...
RequiredQual_token_str      master 's degre manag financ econom at least 5...
JobRequirment_token_str     develop tool system provid critic financi oper...
Name: 11223, dtype: object
('SIMILARITY SCORE: ', 0.4898332490345752)
Title                                                 Chief Financial Officer
JobDescription              Armenian Datacom Company (ADC) is seeking a 

In [18]:
df_job_related.iloc[1]

Title                       Full-time Community Connections Intern (paid i...
JobDescription                                                            nan
RequiredQual                - Bachelor's Degree; Master's is preferred;\n-...
JobRequirment                                                             nan
JobDescription_token                                                    [nan]
RequiredQual_token          [bachelor, 's, degre, master, 's, prefer, exce...
JobRequirment_token                                                     [nan]
JobDescription_token_str                                                  nan
RequiredQual_token_str      bachelor 's degre master 's prefer excel skill...
JobRequirment_token_str                                                   nan
Name: 1, dtype: object

In [19]:
for index, score in find_similar_jobs_tfidf(1):
    print(df_job_related.iloc[index])
    print("SIMILARITY SCORE: ", score)

Title                                                 IT Teacher  (full time)
JobDescription                                                            nan
RequiredQual                - Degree in Computer Science, Information Tech...
JobRequirment                                                             nan
JobDescription_token                                                    [nan]
RequiredQual_token          [degre, comput, scienc, inform, technolog, rel...
JobRequirment_token                                                     [nan]
JobDescription_token_str                                                  nan
RequiredQual_token_str      degre comput scienc inform technolog relat dis...
JobRequirment_token_str                                                   nan
Name: 114, dtype: object
('SIMILARITY SCORE: ', 0.6631658107959908)
Title                       Non-paid part or full time Administrative Intern
JobDescription                                                           na

In [21]:
df_job_related.iloc[2]

Title                                                     Country Coordinator
JobDescription              Public outreach and strengthening of a growing...
RequiredQual                - Degree in environmentally related field, or ...
JobRequirment               - Working with the Country Director to provide...
JobDescription_token        [public, outreach, strengthen, grow, network, ...
RequiredQual_token          [degre, environment, relat, field, 5, year, re...
JobRequirment_token         [work, countri, director, provid, environment,...
JobDescription_token_str    public outreach strengthen grow network enviro...
RequiredQual_token_str      degre environment relat field 5 year relev exp...
JobRequirment_token_str     work countri director provid environment infor...
Name: 2, dtype: object

In [22]:
for index, score in find_similar_jobs_tfidf(2):
    print(df_job_related.iloc[index])
    print("SIMILARITY SCORE: ", score)

Title                                           Country Coordinator - Armenia
JobDescription              CENN - Caucasus Environmental NGO Network - is...
RequiredQual                1. Education: University education in environm...
JobRequirment               -  Write first hand articles, conduct intervie...
JobDescription_token        [cenn, caucasu, environment, ngo, network, loo...
RequiredQual_token          [1, educ, univers, educ, environment, scienc, ...
JobRequirment_token         [write, first, hand, articl, conduct, intervie...
JobDescription_token_str    cenn caucasu environment ngo network look coun...
RequiredQual_token_str      1 educ univers educ environment scienc relat f...
JobRequirment_token_str     write first hand articl conduct interview coll...
Name: 404, dtype: object
('SIMILARITY SCORE: ', 0.45858797345562224)
Title                                                     Country Coordinator
JobDescription              Veya Limited needs an experienced Business Ma

### Using custom Doc2Vec
#### Note: 
This has already been done, so just going to load the model and compute similarity

In [29]:
from gensim.models.doc2vec import Doc2Vec

filename = "C:\\Users\\deeparam\\Desktop\\DMCA\\Sols_DSP_Capstone_Projects_PS_Chegg_2019\\6. TEXT MINING - ONLINE JOB POSTING\\outputs\\doc2vec_model_trained_300"
d2vmodel = Doc2Vec.load(filename)

#### Keeping original columns

In [ ]:
# iterate through each row vs rest of rows to calculate DIS-similarity

job_dissimilarity_d2v_df = pd.DataFrame(columns=['index', 'similar-job', 'dissimilar-score'])

for job1_idx in range(3):  # rows
    for job2_idx in range(1000):
        title_similarity = 1- scipy.spatial.distance.cosine(d2vmodel.infer_vector(df_job_related.iloc[job1_idx]['Title'].split()), d2vmodel.infer_vector(df_job_related.iloc[job2_idx]['Title'].split()))
        jd_similarity = 1- scipy.spatial.distance.cosine(d2vmodel.infer_vector(df_job_related.iloc[job1_idx]['JobDescription'].split()), d2vmodel.infer_vector(df_job_related.iloc[job2_idx]['JobDescription'].split()))
        jr_similarity = 1- scipy.spatial.distance.cosine(d2vmodel.infer_vector(df_job_related.iloc[job1_idx]['JobRequirment'].split()), d2vmodel.infer_vector(df_job_related.iloc[job2_idx]['JobRequirment'].split()))
        rq_similarity = 1- scipy.spatial.distance.cosine(d2vmodel.infer_vector(df_job_related.iloc[job1_idx]['RequiredQual'].split()), d2vmodel.infer_vector(df_job_related.iloc[job2_idx]['RequiredQual'].split()))

        averaged_score = (title_similarity + jd_similarity + jr_similarity + rq_similarity) / 4.0
        job_dissimilarity_d2v_df = job_dissimilarity_d2v_df.append({'index': job1_idx, 'similar-job': job2_idx, 'dissimilar-score': (1-averaged_score)},ignore_index=True)

In [ ]:
job_dissimilarity_d2v_df_sorted = job_dissimilarity_d2v_df.sort_values(by=['index', 'dissimilar-score'], ascending=[True, True])

In [ ]:
job_dissimilarity_d2v_df_sorted.head(10)

In [ ]:
# Show top 5 similar results to job - Chief Financial Officer
similar_jobs = job_dissimilarity_d2v_df_sorted.loc[job_dissimilarity_d2v_df_sorted['index'] == 0].head(6)['similar-job'].tolist()

for job in similar_jobs:
    print(df_job_related.iloc[int(job)])

In [ ]:
# Show top 5 similar results to job - Intern
similar_jobs = job_dissimilarity_d2v_df_sorted.loc[job_dissimilarity_d2v_df_sorted['index'] == 1].head(6)['similar-job'].tolist()

for job in similar_jobs:
    print(df_job_related.iloc[int(job)])

In [ ]:
# Show top 5 similar results to job - Country Coordinator
similar_jobs = job_dissimilarity_d2v_df_sorted.loc[job_dissimilarity_d2v_df_sorted['index'] == 2].head(6)['similar-job'].tolist()

for job in similar_jobs:
    print(df_job_related.iloc[int(job)])

#### Using only the keywords

In [ ]:
# iterate through each row vs rest of rows to calculate DIS-similarity

job_dissimilarity_d2v_df = pd.DataFrame(columns=['index', 'similar-job', 'dissimilar-score'])

for job1_idx in range(3):  # rows
    for job2_idx in range(1000):
        title_similarity = 1- scipy.spatial.distance.cosine(d2vmodel.infer_vector(df_job_related.iloc[job1_idx]['Title'].split()), d2vmodel.infer_vector(df_job_related.iloc[job2_idx]['Title'].split()))
        jd_similarity = 1- scipy.spatial.distance.cosine(d2vmodel.infer_vector(df_job_related.iloc[job1_idx]['JobDescription_keywords'].split()), d2vmodel.infer_vector(df_job_related.iloc[job2_idx]['JobDescription_keywords'].split()))
        jr_similarity = 1- scipy.spatial.distance.cosine(d2vmodel.infer_vector(df_job_related.iloc[job1_idx]['JobRequirment_keywords'].split()), d2vmodel.infer_vector(df_job_related.iloc[job2_idx]['JobRequirment_keywords'].split()))
        rq_similarity = 1- scipy.spatial.distance.cosine(d2vmodel.infer_vector(df_job_related.iloc[job1_idx]['RequiredQual_keywords'].split()), d2vmodel.infer_vector(df_job_related.iloc[job2_idx]['RequiredQual_keywords'].split()))

        averaged_score = (title_similarity + jd_similarity + jr_similarity + rq_similarity) / 4.0
        job_dissimilarity_d2v_df = job_dissimilarity_d2v_df.append({'index': job1_idx, 'similar-job': job2_idx, 'dissimilar-score': (1-averaged_score)},ignore_index=True)

In [ ]:
job_dissimilarity_d2v_df_sorted = job_dissimilarity_d2v_df.sort_values(by=['index', 'dissimilar-score'], ascending=[True, True])

In [ ]:
# Show top 5 similar results to job - Chief Financial Officer
similar_jobs = job_dissimilarity_d2v_df_sorted.loc[job_dissimilarity_d2v_df_sorted['index'] == 0].head(6)['similar-job'].tolist()

for job in similar_jobs:
    print(df_job_related.iloc[int(job)])

In [ ]:
# Show top 5 similar results to job - Intern
similar_jobs = job_dissimilarity_d2v_df_sorted.loc[job_dissimilarity_d2v_df_sorted['index'] == 1].head(6)['similar-job'].tolist()

for job in similar_jobs:
    print(df_job_related.iloc[int(job)])

In [ ]:
# Show top 5 similar results to job - Coordinator
similar_jobs = job_dissimilarity_d2v_df_sorted.loc[job_dissimilarity_d2v_df_sorted['index'] == 2].head(6)['similar-job'].tolist()

for job in similar_jobs:
    print(df_job_related.iloc[int(job)])

In [ ]:
# iterate through each row vs rest of rows to calculate DIS-similarity

job_dissimilarity_d2v_df = pd.DataFrame(columns=['index', 'similar-job', 'dissimilar-score'])

for job1_idx in range(3):  # rows
    for job2_idx in range(1000):
        title_similarity = 1- scipy.spatial.distance.cosine(d2vmodel.infer_vector(df_job_related.iloc[job1_idx]['Title'].split()), d2vmodel.infer_vector(df_job_related.iloc[job2_idx]['Title'].split()))
        jd_similarity = 1- scipy.spatial.distance.cosine(d2vmodel.infer_vector(df_job_related.iloc[job1_idx]['JobDescription'].split()), d2vmodel.infer_vector(df_job_related.iloc[job2_idx]['JobDescription'].split()))
        jr_similarity = 1- scipy.spatial.distance.cosine(d2vmodel.infer_vector(df_job_related.iloc[job1_idx]['JobRequirment'].split()), d2vmodel.infer_vector(df_job_related.iloc[job2_idx]['JobRequirment'].split()))
        rq_similarity = 1- scipy.spatial.distance.cosine(d2vmodel.infer_vector(df_job_related.iloc[job1_idx]['RequiredQual'].split()), d2vmodel.infer_vector(df_job_related.iloc[job2_idx]['RequiredQual'].split()))

        averaged_score = (title_similarity*0.9 + jd_similarity*0.3 + jr_similarity*0.3 + rq_similarity*0.3) / 4.0
        job_dissimilarity_d2v_df = job_dissimilarity_d2v_df.append({'index': job1_idx, 'similar-job': job2_idx, 'dissimilar-score': (1-averaged_score)},ignore_index=True)

In [ ]:
job_dissimilarity_d2v_df_sorted = job_dissimilarity_d2v_df.sort_values(by=['index', 'dissimilar-score'], ascending=[True, True])

In [ ]:
job_dissimilarity_d2v_df_sorted.head(10)

In [ ]:
# Show top 5 similar results to job - Chief Financial Officer
similar_jobs = job_dissimilarity_d2v_df_sorted.loc[job_dissimilarity_d2v_df_sorted['index'] == 0].head(6)['similar-job'].tolist()

for job in similar_jobs:
    print(df_job_related.iloc[int(job)])

In [ ]:
# Show top 5 similar results to job - Intern
similar_jobs = job_dissimilarity_d2v_df_sorted.loc[job_dissimilarity_d2v_df_sorted['index'] == 1].head(6)['similar-job'].tolist()

for job in similar_jobs:
    print(df_job_related.iloc[int(job)])

In [ ]:
# Show top 5 similar results to job - Country Coordinator
similar_jobs = job_dissimilarity_d2v_df_sorted.loc[job_dissimilarity_d2v_df_sorted['index'] == 2].head(6)['similar-job'].tolist()

for job in similar_jobs:
    print(df_job_related.iloc[int(job)])

#To identify the similarity between jobs, the four columns – Title, JobDescription, RequiredQual and JobRequirement were selected as they provide different views of describing jobs. 

To compute job similarity for a field, the chosen measure was Cosine Similarity. The normalised pairwise cosine distance gave the dissimilarity score between jobs and subtracting 1 from a distance gave the similarity score. To compute the overall job similarity, the individual four similarities based on the fields were averaged, which was used to rank jobs by their closeness to each other.

#I used a pre-trained a 'Doc2Vec model' (As it was prepared in other asks) to capture the relationships latent in paragraphs and Gensim package. 

The entire job post of all jobs was given as the train corpus to train the Doc2Vec model, with output embedding dimension of 300, number of epochs=200 and minimum word occurrence size of 2. 
 